# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-17 07:27:49] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-17 07:27:49] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-17 07:27:49] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-17 07:27:52] INFO server_args.py:1646: Attention backend not specified. Use fa3 backend by default.


[2026-01-17 07:27:52] INFO server_args.py:2545: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=80 avail_mem=51.32 GB):  20%|██        | 4/20 [00:00<00:02,  7.66it/s]

Capturing batches (bs=40 avail_mem=51.27 GB):  55%|█████▌    | 11/20 [00:00<00:00, 17.47it/s]

Capturing batches (bs=12 avail_mem=51.25 GB):  70%|███████   | 14/20 [00:01<00:00, 17.84it/s]

Capturing batches (bs=4 avail_mem=51.19 GB):  85%|████████▌ | 17/20 [00:01<00:00, 18.19it/s]

Capturing batches (bs=1 avail_mem=51.18 GB): 100%|██████████| 20/20 [00:01<00:00, 13.79it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  _____. How old are you? ____
A. Tom
B. Tom's
C. Tom's
D. Tom's
Answer: D

A 26-year-old male patient has a history of intermittent lower abdominal pain for more than 2 years, accompanied by symptoms of dysmenorrhea. Based on the clinical presentation, the most likely diagnosis is:
A. Uterine fibroids
B. Ovarian fibroids
C. Uterine polyps
D. Chronic pelvic inflammatory disease
E. Endometriosis
Answer: E

Which of the following is NOT a correct indicator for
Prompt: The president of the United States is
Generated text:  4 feet tall. If the president is standing at a point that is 24 feet away from the center of a clock's face, and the circumference of the clock's face is 72 feet, how many degrees of the clock's face will be visible to the president? Express your answer as a decimal rounded to the nearest tenth. To determine how many degrees of the clock's face the president will see, we need to calculate the angle subtended by the president's po

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I am a [Age] year old [Gender] [Occupation]. I am currently [Current Location] and I am [Current Activity]. I am [Current Hobby or Passion] and I enjoy [Reason for Hobby or Passion]. I am [Current Goal or Motivation] and I am [Current Strength or Weakness]. I am [Current Personality Type] and I am [Current Personality Trait]. I am [Current Personality Style] and I am [Current Personality Style]. I am [Current Personality Type] and I am [Current Personality Trait]. I am [Current Personality Style] and I am [Current Personality

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, the city known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. It is also home to the French Parliament, the French National Museum, and the French Academy of Sciences. Paris is a bustling metropolis with a rich cultural heritage and is a major tourist destination. It is also home to the French Parliament, the French National Museum, and the French Academy of Sciences. The city is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. Paris is a bustling metropolis with a rich cultural heritage and is

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by several key trends:

1. Increased integration with human intelligence: AI is likely to become more integrated with human intelligence, allowing machines to learn from and adapt to human behavior and decision-making processes. This could lead to more sophisticated and adaptive AI systems that can better understand and respond to human emotions and preferences.

2. Enhanced privacy and security: As AI becomes more integrated with human intelligence, there will be increased concerns about privacy and security. AI systems will need to be designed with privacy and security in mind, and there will be a need for robust privacy and security measures to protect user data.

3. Greater emphasis on ethical



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name], and I'm a [career], [industry]. I'm [age] years old, [address], and I currently live [current address]. I have a [current occupation] background and have been in the field for [number] years. I enjoy [what you like to do], and I'm looking to continue my passion for [description of a hobby or activity]. I'm looking forward to [description of your next step]. Thank you for having me. [Name]

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris.

Paris is the capital city of France, known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, and the Marais neighborhood. The city also hosts the annual Le Sejour festival, which showcases classical music and dance performances. Paris has a rich history dating back to ancient times and is a major tourist destination know

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

].

 I

...

:

 AI

 may

 play

 a

In [6]:
llm.shutdown()